In [7]:
import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [ ]:
file_path = 'roblox_games_data.csv'
# Load the latest version
df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "databitio/roblox-games-data",
    file_path
)

In [ ]:
# basic data cleaning (remove duplicate and null data in required row)
start_len = len(df)

df = df.drop_duplicates() #remove duplicate rows

new_len = len(df) - start_len
print(f'{new_len} rows duplicate')

df = df.dropna(subset=['Title','gameID','Active Users','Favorites','Genre']) # remove rows which missing column

new_len = len(df) - start_len - new_len
print(f'{new_len} rows missing required column data')

# reset index
df = df.reset_index(drop=True)

print(f'{len(df) - start_len} total rows removed')
print(f'{len(df)} total rows remaining')

In [ ]:
# List all unique genres
if "Genre" in df.columns:
    genres = df["Genre"].dropna().unique()
    print("All genres:", genres)
else:
    print("No 'Genre' column found.")